In [ ]:
print("ok")

In [ ]:
%pip install langchain
%pip install python-dotenv
%pip show pinecone-client
%pip install langchain-pinecone
%pip install pypdf


In [ ]:
%pip install sentence-transformers
%pip install ctransformers

1. Import Dependecies

In [ ]:
import os
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
# from langchain.llms import CTransformers
from langchain_community.llms import CTransformers


from tqdm import tqdm, tqdm_notebook

2. Load env variables

In [ ]:
load_dotenv()

In [ ]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_ENV=os.getenv("PINECONE_ENV")
EMBEDDING_MODEL_NAME=os.getenv("EMBEDDING_MODEL_NAME")
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

3. Document loading

In [ ]:
# Extract Data from PDF(s)
# https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf
def load_pdf(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader,
                             show_progress=True,
                             )
    document = loader.load()

    return document

4. Extract data

In [ ]:
extracted_data = load_pdf("../data")

In [ ]:
print(f"length of the extracted data: {len(extracted_data)}")

5. Text Splitting / chunking
- https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter

In [ ]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [ ]:
# Calling texts_split function
text_chunks = text_split(extracted_data)

In [28]:
print(f"length of the text_chunks: {len(text_chunks)}")

length of the text_chunks: 10247


7.Text (text chunks) embedding
- A hugging face embedding model is used
- sentence-transformers/all-MiniLM-L6-v2
- https://python.langchain.com/docs/integrations/text_embedding/huggingfacehub

In [16]:
# Download the embedding model
def download_embedding_model():
    embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)
    return embeddings

In [17]:
# Calling download_embedding_model function
embeddings = download_embedding_model()

In [25]:
query_result = embeddings.embed_query("What is the capital of France?")
print(f"length of the query result: {len(query_result)}")

length of the query result: 384


8. Initiate pinecone vector and store data
- https://python.langchain.com/docs/integrations/vectorstores/pinecone

In [27]:
# Initialize Pinecone
#Initializing the Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name=PINECONE_INDEX_NAME

index = pc.Index(PINECONE_INDEX_NAME)
#Creating Embeddings for Each of The Text Chunks & storing
docsearch=PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

9. Generate Rank Results for testing
- https://python.langchain.com/docs/integrations/vectorstores/pinecone

In [29]:
docsearch=PineconeVectorStore.from_existing_index(index_name, embeddings) 

query_result2= "What are the new slabs applicable for income tax in Sri Lanka"

docs=docsearch.similarity_search(query_result2, k=5)

print(docs[0].page_content)

Sri Lanka Accounting Standard – LKAS 12  
 
 
 
Income Taxes


10. Define Prompt Template
- https://python.langchain.com/docs/modules/model_io/prompts/

In [30]:
prompt_template ="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [31]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

11. Defining LLM

In [32]:
llm=CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':2000,
                          'temperature':0.8},)

12. Define the QA chain

In [33]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_type="mmr",search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

13. Chat

In [ ]:
while True:
    user_input = input(f'Input Prompt:')
    result=qa({"query":user_input},)
    print("Question: ",user_input)
    print("Response: ",result["result"])    

14. Delete All Records

# Initialize Pinecone
#Initializing the Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name=PINECONE_INDEX_NAME

index = pc.Index(PINECONE_INDEX_NAME)

index.delete(delete_all=True)